# The main loop

## Look up https://github.com/peterhinch/micropython-async/blob/master/TUTORIAL.md

In [1]:
%websocketconnect 192.168.1.130:8266 --password horsetoe


 ** WebREPL connected **

MicroPython v1.12-375-g28833690b on 2020-04-16; ESP module with ESP8266
Type "help()" for more information.


In [2]:
import os
os.listdir()

['boot.py', 'credentials.json', 'credentials.txt', 'lib', 'send_data.py', 'sensors.py', 'webrepl_cfg.py', 'wifi.py']


In [ ]:
import wifi

In [ ]:
dir(wifi)

In [ ]:
import send_data

In [ ]:
dir(send_data)

In [ ]:
import sensors

In [ ]:
dir(sensors)

In [ ]:
import uasyncio as asyncio
import wifi
import send_data
import sensors
import gc
import collections.deque
import ntptime
import network
import webrepl

In [ ]:
wifi_interface = network.WLAN(network.STA_IF)


In [ ]:
%meminfo

In [ ]:
webrepl.start()
print('network config:', wifi_interface.ifconfig())

In [7]:
import uasyncio as asyncio
import wifi
import send_data
import sensors
import gc
import collections.deque
import ntptime
import network
import webrepl

wifi_interface = network.WLAN(network.STA_IF)

while not wifi_interface.isconnected():
    wifi.connectWifi()

print('network config:', wifi_interface.ifconfig())

data_queue = collections.deque.deque()
get_ID = send_data.generateMessageID()

async def getDataTask(queue):
    while True:
        print("getDataTask")
        result = sensors.readData()
        queue.append(result)
        await asyncio.sleep(10)
        
async def sendDataTask(queue, message_id_generator, url="http://192.168.1.151:9494/station"):
    while True:
        print("sendDataTask")
        send_data.processQueue(queue, message_id_generator, url)
        await asyncio.sleep(30)
    
async def setTimeDateTask():
    print("setTimeDateTask")
    ntptime.settime()
    await asyncio.sleep(86400)

async def checkWifiConnectionTask():
    print("checkWifiConnectionTask")
    wifi.connectWifi()
    await asyncio.sleep(60)
    
async def syncLogDataTask():
    #Send memory info, queue length3
    pass

async def getPowerDataTask():
    pass

async def collectGarbageTask():
    print("collectGarbageTask")
    #http://docs.micropython.org/en/latest/reference/constrained.html#the-heap
    gc.collect()
    gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())
    await asyncio.sleep(600)

def run():
    print('starting weatherstation')

    loop = asyncio.get_event_loop()

    loop.create_task(checkWifiConnectionTask())
    loop.create_task(setTimeDateTask())
    loop.create_task(getDataTask(data_queue))
    loop.create_task(collectGarbageTask()) 
    loop.create_task(sendDataTask(data_queue, get_ID, "http://192.168.1.151:9494/station"))
    loop.run_forever()

run()

network config: ('192.168.1.130', '255.255.255.0', '192.168.1.1', '8.8.8.8')
starting weatherstation
getDataTask
getDataTask
checkWifiConnectionTask
Checking network status
still connected to wifi
setTimeDateTask
getDataTask
collectGarbageTask
sendDataTask


Traceback (most recent call last):
  File "<stdin>", line 70, in <module>
  File "<stdin>", line 68, in run
  File "uasyncio/core.py", line 1, in run_forever
  File "uasyncio/core.py", line 1, in run_until_complete
  File "<stdin>", line 31, in sendDataTask
  File "send_data.py", line 27, in processQueue
  File "send_data.py", line 13, in postData
  File "/lib/urequests.py", line 115, in post
  File "/liurequests.py", line 81, in request
eyboardInterrupt: 



In [5]:
main_script="""import uasyncio as asyncio
import wifi
import send_data
import sensors
import gc
import collections.deque
import ntptime

data_queue = collections.deque.deque()
get_ID = send_data.generateMessageID()

async def getDataTask(queue):
    while True:
        print("getDataTask")
        result = sensors.readData()
        queue.append(result)
        await asyncio.sleep(10)
        
async def sendDataTask(queue, message_id_generator, url="http://192.168.1.151:9494/station"):
    while True:
        print("sendDataTask")
        send_data.processQueue(queue, message_id_generator, url)
        await asyncio.sleep(30)
    
async def setTimeDateTask():
    print("setTimeDateTask")
    ntptime.settime()
    await asyncio.sleep(86400)

async def checkWifiConnectionTask():
    print("checkWifiConnectionTask")
    wifi.connectWifi()
    await asyncio.sleep(60)
    
async def syncLogDataTask():
    pass

async def getPowerDataTask():
    pass

async def collectGarbageTask():
    print("collectGarbageTask")
    #http://docs.micropython.org/en/latest/reference/constrained.html#the-heap
    gc.collect()
    gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())
    await asyncio.sleep(600)

def run():
    print('starting weatherstation')

    loop = asyncio.get_event_loop()

    loop.create_task(getDataTask(data_queue))
    loop.create_task(sendDataTask(data_queue, get_ID, "http://192.168.1.151:9494/station"))
    loop.create_task(setTimeDateTask())
    loop.create_task(checkWifiConnectionTask())
    loop.create_task(collectGarbageTask())
    loop.run_forever()

run()

"""

In [ ]:
os.listdir()

In [ ]:
os.remove('main.py')

In [ ]:
with open('main.py') as f:
    print(f.read())

In [ ]:
with open('main.txt', 'w') as f:
    f.write(main_script)

### Double check the write

In [ ]:
with open('main.txt') as f:
    print(f.read())

## Move temp file to `send_data.py`

In [ ]:
os.rename('main.txt', 'main.py')

In [10]:
%disconnect

Device esp8266 disconnected.